<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h3>Install libs</h3>

In [1]:
#!conda install -c conda-forge geopy --yes

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



<h3>Importing lib</h3>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import requests
import folium
import json
from sklearn import preprocessing

<h3>Importing Data</h3>

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

<h3>Prepare Data</h3>

In [6]:
CLIENT_ID = 'XTCXY3U3C503HMV11BGJGRWXWOBM2DKNFZGZBNA5PO3PVX1P'
CLIENT_SECRET = 'S42YS0GCQHZTC33FKMJRICAMEI5BKSUGJEL14K33L1IS5MUI'
LIMIT = '30'
VERSION = '20180605'

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentails:
CLIENT_ID: XTCXY3U3C503HMV11BGJGRWXWOBM2DKNFZGZBNA5PO3PVX1P
CLIENT_SECRET:S42YS0GCQHZTC33FKMJRICAMEI5BKSUGJEL14K33L1IS5MUI


In [7]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [10]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
map_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
    
map_Bronx

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
def getTrendingVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/trending?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        print(name)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    trending_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    trending_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(trending_venues)

In [ ]:
#Bronx_venues = getTrendingVenues(names=Bronx_data['Neighborhood'],
#                                   latitudes=Bronx_data['Latitude'],
#                                   longitudes=Bronx_data['Longitude']
#                                  )
#Bronx_venues

<h5>Request</h5>
<p>https://api.foursquare.com/v2/venues/trending?&client_id=Y2EX4PQHUQVSBZG5E1APGM21IKCVRIFN2H125GLHVIPA1EZ3&<br/>client_secret=A1ZKOX0A5SNOMFR0BCUJTMHF5MQTGBCNQTFARKVORMUMLJEG&v=20180605&ll=40.894705,-73.847201&radius=500&limit=30</p>
<h5>Response</h5>
<p>{"meta":{"code":429,"errorType":"quota_exceeded","errorDetail":"Quota exceeded","requestId":"5dad4f748afbe0002dcf8263"},"response":{}}</p>

In [16]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )
Bronx_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
2,Wakefield,40.894705,-73.847201,Pitman Deli,40.894149,-73.845748,Food
3,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
4,Co-op City,40.874294,-73.829939,Modell's Sporting Goods,40.872584,-73.829532,Sporting Goods Shop


<p>Adds new Weight importance depending on the venue category</p>

In [59]:
Bronx_venues_df = Bronx_venues[['Venue Latitude','Venue Longitude','Venue Category']]
Bronx_venues_cat_df = Bronx_venues_df['Venue Category']
def getVenueCategories(df):
    Bronx_venues_cat = pd.DataFrame(df['Venue Category'].value_counts()).reset_index()
    Bronx_venues_cat.columns = ['Venue Category','Venue Count']
    return (Bronx_venues_cat[Bronx_venues_cat['Venue Count'] > 4])

Bronx_venues_cat = getVenueCategories(Bronx_venues_df)
Bronx_venues_cat

,Venue Category,Venue Count
0,Pizza Place,35
1,Chinese Restaurant,18
2,Deli / Bodega,14
3,Grocery Store,13
4,Pharmacy,13
5,Bus Station,12
6,Supermarket,12
7,Donut Shop,12
8,Park,9
9,Diner,8


In [60]:
Restaurant = Bronx_venues_df[Bronx_venues_df['Venue Category'].str.contains('Restaurant')]['Venue Category'].values
cat_sec = ['Pizza Place','Donut Shop','Diner','Fried Chicken Joint','Sandwich Place','Bar','Coffee Shop','Food','Deli / Bodega']
Other = Bronx_venues_df[~Bronx_venues_df['Venue Category'].str.contains('Restaurant|Interested')]['Venue Category'].values

Bronx_venues_df = Bronx_venues_df.replace(to_replace = Restaurant,  
                            value ="Restaurant")
Bronx_venues_df = Bronx_venues_df.replace(to_replace = cat_sec,  
                            value ="Interested")
Bronx_venues_df = Bronx_venues_df.replace(to_replace = Other,  
                            value ="Other")

Bronx_venues_cat = getVenueCategories(Bronx_venues_df)
Bronx_venues_cat

,Venue Category,Venue Count
0,Other,215
1,Interested,99
2,Restaurant,73


In [61]:
Bronx_venues_df.head()

,Venue Latitude,Venue Longitude,Venue Category
0,40.894123,-73.845892,Other
1,40.894187,-73.845862,Other
2,40.894149,-73.845748,Interested
3,40.876374,-73.829940,Interested
4,40.872584,-73.829532,Other


In [62]:
map_Bronx_ = folium.Map(location=[latitude, longitude], zoom_start=11)
coloursL = {'Restaurant':'red','Interested':'green','Other':'yellow'}
colours = {'Restaurant':'#F61042','Interested':'#97F235','Other':'#ECE935'}
coloursL['Restaurant']
for lat, lng, label in zip(Bronx_venues_df['Venue Latitude'], Bronx_venues_df['Venue Longitude'], Bronx_venues_df['Venue Category']):
    cL = coloursL[label]
    c = colours[label]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cL,
        fill=True,
        fill_color=c,
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx_)


map_Bronx_

In [64]:
def score_to_numeric(x):
    if x=='Restaurant':
        return 5
    if x=='Interested':
        return 3
    if x=='Other':
        return 2
    else:
        return x

In [65]:
Bronx_venues_df['Venue Category'] = Bronx_venues_df['Venue Category'].apply(score_to_numeric)
Bronx_venues_df.head()

,Venue Latitude,Venue Longitude,Venue Category
0,40.894123,-73.845892,2
1,40.894187,-73.845862,2
2,40.894149,-73.845748,3
3,40.876374,-73.829940,3
4,40.872584,-73.829532,2


<h3>Methods</h3>

In [97]:
# set number of clusters
kclusters = 6
_clustering = Bronx_venues_df

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,init='random',n_init=20,  random_state=0).fit(_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 1, 5, 1, 1, 0, 0], dtype=int32)

In [98]:
sorted_areas = pd.DataFrame()
sorted_areas.insert(0, 'Cluster Labels', kmeans.labels_)
sorted_areas['Venue'] = Bronx_venues['Venue']
sorted_areas.head()

,Cluster Labels,Venue
0,1,Lollipops Gelato
1,1,Shell
2,0,Pitman Deli
3,0,Capri II Pizza
4,1,Modell's Sporting Goods


In [99]:
_merged = Bronx_venues
_merged = _merged.join(sorted_areas.set_index('Venue'), on='Venue')
_merged.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,1
1,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station,1
1,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station,1
1,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station,1
1,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station,1
2,Wakefield,40.894705,-73.847201,Pitman Deli,40.894149,-73.845748,Food,0
3,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place,0
4,Co-op City,40.874294,-73.829939,Modell's Sporting Goods,40.872584,-73.829532,Sporting Goods Shop,1
4,Co-op City,40.874294,-73.829939,Modell's Sporting Goods,40.872584,-73.829532,Sporting Goods Shop,2
5,Co-op City,40.874294,-73.829939,Townhouse Restaurant,40.876086,-73.828868,Restaurant,5


<h3>Results</h3>

In [100]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(_merged['Venue Latitude'], _merged['Venue Longitude'], _merged['Venue'], _merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<p>Following cluser centroids are the better locations to launch the new Restaurant</p>

In [102]:
results = pd.DataFrame(kmeans.cluster_centers_)
results.columns = ['Latitude','Longitude','W']
results

,Latitude,Longitude,W
0,40.856934,-73.873055,3.0
1,40.852289,-73.841135,2.0
2,40.872983,-73.895667,2.0
3,40.846258,-73.902155,5.0
4,40.827599,-73.905848,2.0
5,40.852510,-73.843539,5.0


In [103]:
map_result = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(results['Latitude'], results['Longitude'], 'Good Location'):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_result)  
    
map_result

<p>These locations are the sugested locations</p>